In [43]:
import pandas as pd
import numpy as np
import unicodedata
import re

def norm_names(x):
    y1 = re.sub('\\t','t',unicodedata.normalize('NFKD', x).encode('ASCII', 'ignore').decode()).strip().lower()
    y2 = re.sub('\\n', 'n', y1).strip()
    z = re.sub('[\W_]','',y2.replace(' ', '').replace('/',''))
    return z   

In [44]:
country = 'Ireland'

In [45]:
#put your files paths/names in a list here. Index 0=spon_info, 1=normalized_names, 2=trials
file_names = ['/Users/nicholasdevito/Desktop/new country pulls/nov2020_spon_info.csv', 
              '/Users/nicholasdevito/Desktop/new country pulls/normalized_sponsor_names_nov2020.xlsx', 
              '/Users/nicholasdevito/Desktop/new country pulls/trials_nov.csv']
#country_name is a string

def get_con_info(file_names, country_name):
    spon_df = pd.read_csv(file_names[0]).drop('Unnamed: 0', axis=1)
    norm = pd.read_excel(file_names[1])
    trials_csv = pd.read_csv(file_names[2])
    trials = trials_csv[['trial_id', 'number_of_countries', 'has_results', 'results_expected']].drop_duplicates()
    sample = spon_df[(spon_df.sponsor_country == country_name)].reset_index(drop=True)
    sample1 = sample.drop_duplicates()
    merged = sample.merge(trials, on='trial_id', how='left').fillna('')
    merged['name_of_sponsor_lower'] = (merged.sponsor_name.apply(norm_names))
    norm_s = norm[['name_of_sponsor', 'normalized_name']].reset_index(drop=True).fillna('')
    norm_s['name_of_sponsor_lower'] = (norm_s.name_of_sponsor.apply(norm_names))
    with_norm = merged.merge(norm_s, on='name_of_sponsor_lower', how='left', indicator=True)
    no_sponsor = with_norm[(with_norm.name_of_sponsor_lower.str.contains('nodata')) | 
                           (with_norm.name_of_sponsor_lower == '')].index.values.tolist()
    for n in no_sponsor:
        with_norm.at[n,'normalized_name'] = 'No Sponsor Name Given'
        with_norm.at[n,'_merge'] = 'both'
    return with_norm

first_df = get_con_info(file_names, country)

In [46]:
errors = first_df[first_df._merge == 'left_only'].index.values.tolist()
errors

[]

In [47]:
first_df.iloc[errors]

,trial_id,protocol_country,sponsor_name,sponsor_country,sponsor_status,number_of_countries,has_results,results_expected,name_of_sponsor_lower,name_of_sponsor,normalized_name,_merge


In [48]:
first_df.head()

,trial_id,protocol_country,sponsor_name,sponsor_country,sponsor_status,number_of_countries,has_results,results_expected,name_of_sponsor_lower,name_of_sponsor,normalized_name,_merge
0,2007-007804-14,IE,Coombe Women's Hospital,Ireland,Non-Commercial,1,0,0,coombewomenshospital,Coombe Women's Hospital,Coombe Women's Hospital,both
1,2009-010918-30,IE,"Amar Agha, Beaumont Hospital",Ireland,Non-Commercial,1,0,0,amaraghabeaumonthospital,"Amar Agha, Beaumont Hospital",Beaumont Hospital,both
2,2011-000591-34,IE,Mater University Hospital,Ireland,Non-Commercial,1,0,0,materuniversityhospital,Mater University Hospital,Mater University Hospital,both
3,2011-002561-38,IE,St. Vincent's University Hospital,Ireland,Non-Commercial,1,0,0,stvincentsuniversityhospital,St. Vincent's University Hospital,St. Vincent's University Hospital,both
4,2011-001180-53,CZ,Janssen R&D Ireland,Ireland,Commercial,15,1,0,janssenrdireland,"Janssen R&D, Ireland",Johnson & Johnson,both


In [49]:
def final_data(df, summary=True):
    df.rename(columns={'sponsor_name': 'euctr_name'}, inplace=True)
    almost_df = df[['trial_id','sponsor_country', 'normalized_name', 'sponsor_status', 'results_expected', 
                           'has_results']]
    final_df = almost_df[(almost_df.has_results.notnull()) & (almost_df.has_results != '')].reset_index(drop=True)
    final_df['results_expected'] = final_df.results_expected.astype(int)
    final_df['has_results'] = final_df.has_results.astype(int)
    final_df['due_with_results'] = np.where((final_df.has_results == 1) & (final_df.results_expected == 1), 1,0)
    trial_count = final_df[['trial_id', 'normalized_name']].drop_duplicates().groupby('normalized_name').count()
    final_df['commercial_status'] = np.where(final_df['sponsor_status'] == 'Commercial',1,0)
    final_df['non-commercial_stauts'] = np.where(final_df['sponsor_status'] == 'Non-Commercial',1,0)
    final_df = final_df.drop_duplicates().reset_index(drop=True)
    if summary:
        summ_df = final_df.groupby('normalized_name').sum()
        summ_df = summ_df.merge(trial_count, on='normalized_name').rename(columns={'trial_id': 'trial_count'})
        return summ_df
    elif not summary:
        return final_df

final_data(first_df).to_csv('{}_summary_counts_nov2020.csv'.format(country))
final_data(first_df, summary=False).to_csv('combined_sponsor_info_{}_nov2020.csv'.format(country))

/Users/nicholasdevito/anaconda3/lib/python3.7/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/Users/nicholasdevito/anaconda3/lib/python3.7/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)


In [ ]:
testing = pd.read_json('/Users/nicholasdevito/Desktop/all_sponsors.json-2.txt')

In [ ]:
testing[(testing.total_trials >= 10) & (testing.total_trials <= 50)].to_csv('/Users/nicholasdevito/Desktop/testing.csv')

In [ ]:
testing.parents[0]